In [ ]:
import pandas as pd
import pandas as pd
import numpy as np
import tensorflow as tf
import torch
from transformers import BertTokenizer, BertModel
import logging

Here, choose a file to create BERT vector embeddings:

In [ ]:
dataset = pd.read_excel("Gly_Human_dhit40_Equal.xlsx", header = None, names = ["Sequence_with_len_21_positive", "Sequence_with_len_21_negative"])
#dataset = pd.read_excel("Crot_Human_cdhit40_Equal.xlsx", header = None, names = ["Sequence_with_len_21_positive", "Sequence_with_len_21_negative"])
#dataset = pd.read_excel("Ubi_Human_cdhit_Equal.xlsx", header = None, names = ["Sequence_with_len_21_positive", "Sequence_with_len_21_negative"])
#dataset = pd.read_excel("Suc_yeast_cdhit40_Equal.xlsx", header = None, names = ["Sequence_with_len_21_positive", "Sequence_with_len_21_negative"])
#dataset = pd.read_excel("Suc_Mouse_cdhit40_Equal.xlsx", header = None, names = ["Sequence_with_len_21_positive", "Sequence_with_len_21_negative"])
#dataset = pd.read_excel("Suc_Human_cdhit40_Equal.xlsx", header = None, names = ["Sequence_with_len_21_positive", "Sequence_with_len_21_negative"])

In [ ]:
print(dataset.head(3))

dataset_sd = dataset

for i in range(0, len(dataset_sd)):
    dataset_sd.iloc[i][0] = dataset_sd.iloc[i][0].replace("", " ")
    dataset_sd.iloc[i][1] = dataset_sd.iloc[i][1].replace("", " ")

print("length of dataset_sd:", len(dataset_sd))

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

##################### Vector Creation for Positives ########################

n = "[CLS]" +  dataset_sd.iloc[0][0] + "[SEP]"

print(n)

positive_sentences = []
for i in range(0, len(dataset_sd)):
    positive_sentences.append("[CLS]" + dataset_sd.iloc[i][0] + "[SEP]")

positive_sentences_tokenized = []
for i in range(0, len(positive_sentences)):
    tokenized_text = tokenizer.tokenize(str(positive_sentences[i]))
    positive_sentences_tokenized.append(tokenized_text)
positive_sentences_tokenized[0:4]

print("Total row number:", len(positive_sentences_tokenized))
print("Sequence length:", len(positive_sentences_tokenized[0]))

positive_sentences_indexes = []
for i in range(0, len(positive_sentences)):
    positive_sentences_indexes.append(tokenizer.convert_tokens_to_ids(positive_sentences_tokenized[i]))

positive_sentences_segment_ids = []

for i in range(0, int(len(positive_sentences)/2)):
    positive_sentences_segment_ids.append([1] * len(positive_sentences_tokenized[0]))
    positive_sentences_segment_ids.append([0] * len(positive_sentences_tokenized[0]))

positive_tokens_tensor = torch.tensor([positive_sentences_indexes])

print("positive_tokens_tensor shape:", positive_tokens_tensor.shape)

positive_segments_tensors = torch.tensor([positive_sentences_segment_ids])
print(positive_segments_tensors)

print("positive_segments_tensors shape:", positive_segments_tensors.shape)

model_positive = BertModel.from_pretrained("bert-base-uncased", output_hidden_states = True)
model_positive.eval()

outputs_positive = []
with torch.no_grad():
    outputs_positive = model_positive(positive_tokens_tensor[0], positive_segments_tensors[0])
    positive_hidden_states = outputs_positive[2]

print ("Number of layers:", len(positive_hidden_states), "  (initial embeddings + 12 BERT layers)")
layer_i = 0

print ("Number of batches:", len(positive_hidden_states[layer_i]))
batch_i = 0

print ("Number of tokens:", len(positive_hidden_states[layer_i][batch_i]))
token_i = 0

print ("Number of hidden units:", len(positive_hidden_states[layer_i][batch_i][token_i]))

print("positive_hidden_states[0].shape:", positive_hidden_states[0].shape)


##################### Vector Creation for Negatives ########################

negative_sentences = []
for i in range(0, len(dataset_sd)):
    negative_sentences.append("[CLS]" + dataset_sd.iloc[i][1] + "[SEP]")

negative_sentences_tokenized = []
for i in range(0, len(negative_sentences)):
    tokenized_text = tokenizer.tokenize(str(negative_sentences[i]))
    negative_sentences_tokenized.append(tokenized_text)

print("Total row number:", len(negative_sentences_tokenized))
print("Sequence length:", len(negative_sentences_tokenized[0]))

negative_sentences_indexes = []
for i in range(0, len(negative_sentences)):
    negative_sentences_indexes.append(tokenizer.convert_tokens_to_ids(negative_sentences_tokenized[i]))

negative_sentences_segment_ids = []

for i in range(0, int(len(negative_sentences)/2)):
    negative_sentences_segment_ids.append([1] * len(negative_sentences_tokenized[0]))
    negative_sentences_segment_ids.append([0] * len(negative_sentences_tokenized[0]))

print("len(negative_sentences_segment_ids):", len(negative_sentences_segment_ids))

negative_tokens_tensor = torch.tensor([negative_sentences_indexes])
negative_tokens_tensor

negative_segments_tensors = torch.tensor([negative_sentences_segment_ids])

model_negative = BertModel.from_pretrained("bert-base-uncased", output_hidden_states = True)
model_negative.eval()

outputs_negative = []
with torch.no_grad():
    outputs_negative = model_negative(negative_tokens_tensor[0], negative_segments_tensors[0])
    negative_hidden_states = outputs_negative[2]

print("outputs_negative[0].shape:", outputs_negative[0].shape)

print ("Number of layers:", len(negative_hidden_states), "  (initial embeddings + 12 BERT layers)")
layer_i = 0

print ("Number of batches:", len(negative_hidden_states[layer_i]))
batch_i = 0

print ("Number of tokens:", len(negative_hidden_states[layer_i][batch_i]))
token_i = 0

print ("Number of hidden units:", len(negative_hidden_states[layer_i][batch_i][token_i]))

print ("CNN Preparation Part started!")

##################### CNN Preparation Part ########################

#### Positive part:

dataset_pos = np.array([])

for j in range(0, len(dataset_sd)):
    #print ("j1:", j)
    a = np.array([])
    for i in range(0,23):
        b = (np.array(positive_hidden_states[12][j][i]) + np.array(positive_hidden_states[11][j][i]) + np.array(positive_hidden_states[10][j][i]) + np.array(positive_hidden_states[9][j][i])) / 4
        a = np.hstack((a,b))
    if len(dataset_pos) == 0:
        dataset_pos = a
    else:
        dataset_pos = np.vstack((dataset_pos, a))
dataset_pos.shape
dataset_pos = pd.DataFrame(dataset_pos)


#### Negative part:

dataset_neg = np.array([])

for j in range(0, len(dataset_sd)):
    #print ("j2:", j)
    a = np.array([])
    for i in range(0,23):
        b = (np.array(negative_hidden_states[12][j][i]) + np.array(negative_hidden_states[11][j][i]) + np.array(negative_hidden_states[10][j][i]) + np.array(negative_hidden_states[9][j][i])) / 4
        a = np.hstack((a,b))
    if len(dataset_neg) == 0:
        dataset_neg = a
    else:
        dataset_neg = np.vstack((dataset_neg, a))
dataset_neg.shape
dataset_neg = pd.DataFrame(dataset_neg)

print("Dataset positive:", dataset_pos.shape)
print("Dataset negative:", dataset_neg.shape)

dataset_pos_labels = [1] * len(dataset_sd)
dataset_neg_labels = [0] * len(dataset_sd)

x = pd.concat([dataset_pos, dataset_neg], ignore_index = True)
y = dataset_pos_labels + dataset_neg_labels
print("Data type of x: ",type(x))
print("Data type of y: ", type(y))

print("x.shape", x.shape)

print("len(y):", len(y))


  Sequence_with_len_21_positive Sequence_with_len_21_negative
0         SKRILPLTVCKMHVLHCQGRN         IIDNWKYHKTKVASYWLIKLD
1         DLIASSGDIIKVSAAGKEALP         ASTTITNIPMKEEQHANTSAN
2         IEEIKSGRLSKELDLVSHHVR         QVWDPQCAPRKDRQVHLTHFE
length of homo_sapiens_sd: 3342


Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

[CLS] S K R I L P L T V C K M H V L H C Q G R N [SEP]
Total row number: 3342
Sequence length: 23
positive_tokens_tensor shape: torch.Size([1, 3342, 23])
tensor([[[1, 1, 1,  ..., 1, 1, 1],
         [0, 0, 0,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 1, 1, 1],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 1, 1, 1],
         [0, 0, 0,  ..., 0, 0, 0]]])
positive_segments_tensors shape: torch.Size([1, 3342, 23])


Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Number of layers: 13   (initial embeddings + 12 BERT layers)
Number of batches: 3342
Number of tokens: 23
Number of hidden units: 768
positive_hidden_states[0].shape: torch.Size([3342, 23, 768])
Total row number: 3342
Sequence length: 23
len(negative_sentences_segment_ids): 3342


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


outputs_negative[0].shape: torch.Size([3342, 23, 768])
Number of layers: 13   (initial embeddings + 12 BERT layers)
Number of batches: 3342
Number of tokens: 23
Number of hidden units: 768
CNN Preparation Part started!
Dataset positive: (3342, 17664)
Dataset negative: (3342, 17664)
Data type of x:  <class 'pandas.core.frame.DataFrame'>
Data type of y:  <class 'list'>
x.shape (6684, 17664)
len(y): 6684


Define file locations to send vector embeddings generated by BERT:

In [ ]:
### Sending vector embeddings generated by BERT for the dataset with 208946 sample sizes:

directory_x = r"/content/drive/MyDrive/Trans-Ubiquitination-Colab/BraveHeart/homo_gly_embeddings_X.npy"
np.save(directory_x, x)
directoy_y = r"/content/drive/MyDrive/Trans-Ubiquitination-Colab/BraveHeart/homo_gly_embeddings_Y.npy"
np.save(directoy_y, y)

#directory_x = r"/content/drive/MyDrive/Trans-Ubiquitination-Colab/BraveHeart/homo_cro_embeddings_X.npy"
#np.save(directory_x, x)
#directoy_y = r"/content/drive/MyDrive/Trans-Ubiquitination-Colab/BraveHeart/homo_cro_embeddings_Y.npy"
#np.save(directoy_y, y)

#directory_x = r"/content/drive/MyDrive/Trans-Ubiquitination-Colab/Ubi Human Equal Models/homo_ubi_embeddings20000_1_X.npy"
#np.save(directory_x, x)
#directoy_y = r"/content/drive/MyDrive/Trans-Ubiquitination-Colab/Ubi Human Equal Models/homo_ubi_embeddings20000_1_Y.npy"
#np.save(directoy_y, y)

#directory_x = r"/content/drive/MyDrive/Trans-Ubiquitination-Colab/BraveHeart/yeast_suc_embeddings_X.npy"
#np.save(directory_x, x)
#directoy_y = r"/content/drive/MyDrive/Trans-Ubiquitination-Colab/BraveHeart/yeast_suc_embeddings_Y.npy"
#np.save(directoy_y, y)

#directory_x = r"/content/drive/MyDrive/Trans-Ubiquitination-Colab/BraveHeart/mouse_suc_embeddings_X.npy"
#np.save(directory_x, x)
#directoy_y = r"/content/drive/MyDrive/Trans-Ubiquitination-Colab/BraveHeart/mouse_suc_embeddings_Y.npy"
#np.save(directoy_y, y)

#directory_x = r"/content/drive/MyDrive/Trans-Ubiquitination-Colab/BraveHeart/homo_suc_embeddings_X.npy"
#np.save(directory_x, x)
#directoy_y = r"/content/drive/MyDrive/Trans-Ubiquitination-Colab/BraveHeart/homo_suc_embeddings_Y.npy"
#np.save(directoy_y, y)